In [16]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [17]:
df = pd.read_pickle("data/clean_data/CA_full_data_filtered.pkl")

data = df.TXT.values.tolist()

In [18]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [19]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [20]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'model', 'Use', 'Base', 'base', 'USE'])

In [21]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [22]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser'])


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [23]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [24]:
from collections import Counter

frequencies = Counter()
frequencies_ngram = Counter()
stuff = [[(id2word[id], freq) for id, freq in cp] for cp in corpus]
for i, tok in enumerate(stuff):
    for tup in tok:
        if "_" in tup[0]:
            frequencies_ngram[tup[0]] += 1
        frequencies[tup[0]] +=1

In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [26]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"system" + 0.024*"learn" + 0.017*"adaptive" + 0.016*"learning" + '
  '0.013*"environment" + 0.013*"sensor" + 0.011*"sensing" + '
  '0.011*"performance" + 0.011*"propose" + 0.010*"behavior"'),
 (1,
  '0.047*"tree" + 0.033*"power" + 0.029*"reservoir" + 0.027*"results_indicate" '
  '+ 0.020*"electricity" + 0.020*"recognition" + 0.019*"better_performance" + '
  '0.019*"prevalent" + 0.017*"renewable_resource" + 0.017*"transportation"'),
 (2,
  '0.047*"model" + 0.028*"use" + 0.023*"predict" + 0.021*"prediction" + '
  '0.017*"performance" + 0.016*"study" + 0.013*"value" + 0.012*"result" + '
  '0.011*"compare" + 0.011*"high"'),
 (3,
  '0.135*"drought" + 0.042*"fire" + 0.039*"renewable" + '
  '0.034*"machine_learning_base" + 0.021*"heat" + 0.016*"carbon_dioxide" + '
  '0.015*"feature_extraction" + 0.014*"smoke" + 0.012*"spi" + 0.011*"weed"'),
 (4,
  '0.120*"forecast" + 0.110*"forecasting" + 0.065*"co_emission" + 0.042*"mape" '
  '+ 0.027*"decomposition" + 0.021*"mean_absolute_perc

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.574057387794026

Coherence Score:  0.4138809320187076


In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/home/kevin-main/anaconda3/envs/datascience/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.230184 -0.038304       1        1  30.829441
2      0.229079  0.097765       2        1  17.791490
0      0.101604 -0.196733       3        1  13.180381
7      0.146757  0.174153       4        1  12.318824
5      0.140221  0.140140       5        1  10.808421
8      0.026004 -0.179854       6        1   8.047898
6     -0.122775 -0.261380       7        1   4.099675
1     -0.251581  0.075332       8        1   1.282864
4     -0.254521  0.103683       9        1   1.063136
3     -0.244971  0.085198      10        1   0.577869, topic_info=                          Term         Freq        Total Category  logprob  \
325                      model  4121.000000  4121.000000  Default  30.0000   
209                     system  2693.000000  2693.000000  Default  29.0000   
215                       area  2254.000000  2254.000000  Default  28.0000   
14                       datum  4971.000000  4971.000000  Default  27.0000   
114                    predict  1963.000000  1963.000000  Default  26.0000   
...                        ...          ...          ...      ...      ...   
2188                      burn    12.757512    13.643918  Topic10  -5.3918   
5397   based_artificial_neural    12.451823    13.338489  Topic10  -5.4161   
2628                  robustly    12.389724    13.276756  Topic10  -5.4211   
1079                     right    11.068901    11.955470  Topic10  -5.5338   
13047     hydrological_drought    10.334036    11.223925  Topic10  -5.6025   

       loglift  
325    30.0000  
209    29.0000  
215    28.0000  
14     27.0000  
114    26.0000  
...        ...  
2188    5.0864  
5397    5.0848  
2628    5.0844  
1079    5.0765  
13047   5.0710  

[527 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1556      1  0.337334          able
1556      3  0.659532          able
6950      5  0.995783        abrupt
3982      9  0.947756    accordance
1363      4  0.992684  accumulation
...     ...       ...           ...
355       1  0.074857          year
355       2  0.416741          year
355       4  0.018994          year
355       5  0.489364          year
445       2  0.997354         yield

[793 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 1, 8, 6, 9, 7, 2, 5, 4])